<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b2b6520c2577fa2e28461dfdfbaabc77b531681bb5a54c59f37fb827432f631b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 16:12:13
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -34.84 K (-0.24%)
Current PnL: -23.02 L (-15.08%)
CY Booked + Current PnL: -9.13 L (-5.98%)
-------------------
Total profit:  1.62 L
Total loss:  -24.64 L
-------------------
Total Booked + Current PnL: 17.85 L (14.21%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.43 L (61.94%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.21,-8.63,9.46,0.02,15089.0,-15058.0,159501.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.74,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.59,-16.82,20.33,0.09,17042.0,-16950.0,83827.0,94.66,36.0,M-SC,3.20,253.0,-0.99,0.59,10.13,OX40N,NTT,DURABLES
43,ITC,452.00,-0.00,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.32,5.0,-0.11,1.38,4.27,X40,NTT,FMCG
35,ICICIGI,2252.93,1.06,6.24,13.13,20.19,24317.0,10880.0,185200.0,-16.73,49.0,X-MC,7.16,68.0,0.45,1.30,22.30,X40,ATH,INSURANCE
51,MASFIN,398.61,1.80,-3.75,26.80,22.05,25274.0,-3675.0,94305.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.08,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,5.32,-0.44,49.28,48.63,98103.0,-882.0,199072.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,11.06,XR,NTT,MISC
81,VALIANTORG,838.00,4.20,-45.39,218.87,74.15,195565.0,-74253.0,89352.0,-260.95,36.0,H-SC,18.18,149.0,-0.38,0.63,14.53,XR,NTT,CHEMICALS
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.67,3.22,XY24,NTT,AUTO
51,MASFIN,398.61,1.80,-3.75,26.80,22.05,25274.0,-3675.0,94305.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.08,XR,ATH,FINANCE
64,SFL,1287.00,1.71,-39.95,110.88,26.63,174588.0,-104761.0,157457.0,19.19,38.0,M-SC,6.60,239.0,-0.60,1.10,2.90,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-7.63,-20.64,129.22,81.92,117730.0,-23690.0,91108.0,-53.49,20.0,M-SC,2.98,236.0,-0.20,0.64,7.86,XR,NTT,DURABLES
27,GLAXO,3466.20,-2.88,0.17,38.90,39.15,77657.0,348.0,199632.0,-19.47,40.0,X-MC,10.56,60.0,0.00,1.40,29.23,X40,ATH,PHARMA
11,BATAINDIA,2096.00,-2.52,-37.17,115.20,35.20,93130.0,-47828.0,80842.0,5.99,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-2.39,-8.62,37.96,26.07,56796.0,-14120.0,149620.0,-32.65,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
28,GREENPANEL,537.00,-2.13,-34.99,120.18,43.14,143036.0,-64060.0,119018.0,166.80,27.0,M-SC,6.34,240.0,-0.45,0.83,11.95,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.90,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,40.0,H-SC,2.41,139.0,0.00,0.87,23.57,AR,ATH,MISC
37,IEX,219.00,5.32,-0.44,49.28,48.63,98103.0,-882.0,199072.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,11.06,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-4.96,118.76,107.91,166650.0,-7319.0,140325.0,-22.58,36.0,M-SC,10.84,216.0,-0.04,0.98,0.34,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.21,-8.63,9.46,0.02,15089.0,-15058.0,159501.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.74,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.59,-16.82,20.33,0.09,17042.0,-16950.0,83827.0,94.66,36.0,M-SC,3.20,253.0,-0.99,0.59,10.13,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.85,-22.53,46.48,13.47,97096.0,-60768.0,208899.0,-68.12,33.0,H-SC,1.77,158.0,-0.63,1.46,6.22,XY24,NTT,PAINTS
67,SIS,528.00,-1.96,-25.86,63.57,21.26,52327.0,-28718.0,82314.0,1939.90,41.0,H-SC,3.76,166.0,-0.55,0.58,11.21,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.3,2.65,67.81,72.27,117952.0,4497.0,173945.0,-11.98,60.0,M-LC,3.50,99.0,0.04,1.22,8.99,XR,NTT,IT
38,INDIAMART,4810.62,0.9,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,40.0,H-SC,2.41,139.0,0.00,0.87,23.57,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.30,2.65,67.81,72.27,117952.0,4497.0,173945.0,-11.98,60.0,M-LC,3.50,99.0,0.04,1.22,8.99,XR,NTT,IT
38,INDIAMART,4810.62,0.90,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,40.0,H-SC,2.41,139.0,0.00,0.87,23.57,AR,ATH,MISC
37,IEX,219.00,5.32,-0.44,49.28,48.63,98103.0,-882.0,199072.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,11.06,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.64,-1.25,37.35,35.63,77298.0,-2619.0,206955.0,-14.97,42.0,H-MC,3.70,119.0,-0.03,1.45,15.21,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.17,-4.96,118.76,107.91,166650.0,-7319.0,140325.0,-22.58,36.0,M-SC,10.84,216.0,-0.04,0.98,0.34,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.52,-37.17,115.20,35.20,93130.0,-47828.0,80842.0,5.99,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.55,-14.53,73.70,48.45,126295.0,-29142.0,171364.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-2.39,-8.62,37.96,26.07,56796.0,-14120.0,149620.0,-32.65,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
46,JSWINFRA,342.00,0.20,-8.73,26.25,15.23,48000.0,-17490.0,182857.0,-21.89,31.0,X-MC,6.90,66.0,-0.36,1.28,20.71,X40N,NTT,REALTY
8,AWL,485.00,-1.43,-19.20,89.97,53.50,219358.0,-57939.0,243812.0,-60.95,33.0,X-MC,3.06,58.0,-0.26,1.71,8.29,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.00,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.32,5.0,-0.11,1.38,4.27,X40,NTT,FMCG
3,ACC,3906.0,-0.01,-22.23,111.08,64.15,205554.0,-52901.0,185050.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.30,3.80,XY24,BTT,CEMENT
15,CAMS,4762.0,0.98,2.41,21.85,24.79,56359.0,6079.0,257935.0,-2.36,51.0,X-SC,1.87,86.0,0.11,1.81,26.11,X40N,NTT,MISC
66,SIEMENS,4671.5,0.40,-11.06,41.12,25.51,68058.0,-20585.0,165510.0,1.56,65.0,H-LC,2.04,49.0,-0.30,1.16,21.97,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.29,3.04,27.10,30.97,49353.0,5376.0,182114.0,-21.15,55.0,X-MC,2.13,75.0,0.11,1.28,15.98,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.52,-37.17,115.20,35.20,93130.0,-47828.0,80842.0,5.99,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.55,-14.53,73.70,48.45,126295.0,-29142.0,171364.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-2.39,-8.62,37.96,26.07,56796.0,-14120.0,149620.0,-32.65,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.24,-4.49,40.05,33.76,76818.0,-9015.0,191805.0,-32.64,34.0,X-MC,4.06,57.0,-0.12,1.34,0.47,X40,ATH,FMCG
19,COLPAL,3726.84,-0.72,-18.25,73.11,41.51,157399.0,-48075.0,215290.0,-2.56,39.0,X-MC,7.22,61.0,-0.31,1.51,1.03,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.95,-28.19,98.91,42.84,46173.0,-18324.0,46682.0,-53.17,41.0,X-SC,37.53,83.0,-0.40,0.33,1.58,XY24,NTT,MISC
60,RELAXO,1176.00,-0.40,-47.21,188.52,52.31,144461.0,-68531.0,76629.0,-43.76,36.0,X-SC,4.83,91.0,-0.47,0.54,2.57,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-2.52,-37.17,115.20,35.20,93130.0,-47828.0,80842.0,5.99,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.24,-6.45,29.97,21.58,39289.0,-9045.0,131094.0,-52.80,35.0,X-MC,6.23,56.0,-0.23,0.92,16.06,X40N,ATH,IT
52,MEDANTA,1486.00,-0.01,4.03,19.27,24.08,25449.0,5115.0,132065.0,-4.36,47.0,X-SC,5.93,89.0,0.20,0.93,24.44,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,-0.13,-14.86,40.10,19.28,118111.0,-51416.0,294540.0,-25.89,57.0,X-LC,7.02,1.0,-0.44,2.06,8.89,X40,ATH,IT
41,INFY,2275.00,0.25,6.47,45.46,54.87,151442.0,20244.0,333132.0,-17.09,61.0,X-LC,2.29,2.0,0.13,2.33,15.29,X40,BTT,IT
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.67,3.22,XY24,NTT,AUTO
82,VBL,671.64,0.54,-2.20,38.73,35.67,119632.0,-6954.0,308888.0,-13.89,63.0,X-LC,5.48,4.0,-0.06,2.16,11.35,X40N,ATH,FMCG
43,ITC,452.00,-0.00,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.32,5.0,-0.11,1.38,4.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.75,-15.28,114.43,81.67,90948.0,-14331.0,79479.0,7144.44,43.0,L-SC,19.30,271.0,-0.16,0.56,53.91,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.59,-11.99,58.55,39.54,52128.0,-12128.0,89031.0,-31.16,65.0,M-SC,6.69,220.0,-0.23,0.62,25.54,AR,ATH,AUTO
51,MASFIN,398.61,1.80,-3.75,26.80,22.05,25274.0,-3675.0,94305.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.08,XR,ATH,FINANCE
70,SURYODAY,216.00,0.72,-18.93,51.76,23.03,75144.0,-33894.0,145177.0,57.89,50.0,H-SC,9.71,167.0,-0.45,1.02,44.04,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.21,-8.63,9.46,0.02,15089.0,-15058.0,159501.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.74,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.21,-8.63,9.46,0.02,15089.0,-15058.0,159501.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.74,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.24,0.23,20.67,20.95,52161.0,581.0,252349.0,-0.77,73.0,X-LC,15.77,31.0,0.01,1.77,35.95,X40,ATH,PAINTS
50,LTIM,7201.88,0.92,12.48,17.06,31.66,47233.0,30711.0,276862.0,8.03,69.0,H-LC,13.98,43.0,0.65,1.94,53.62,X200,ATH,IT
32,HCLTECH,1922.01,1.15,6.62,16.99,24.73,43823.0,16011.0,257935.0,11.49,65.0,X-LC,4.89,7.0,0.37,1.81,22.86,X40,ATH,IT
40,INDUSINDBK,1800.00,-1.33,-37.46,112.48,32.89,54314.0,-28918.0,48288.0,-696.43,60.0,L-MC,5.80,259.0,-0.53,0.34,32.98,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.86
2,50,75.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    29.11
LC    25.51
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.74
X40      21.24
X40N     12.03
XR        9.61
XY25      9.10
AR        9.06
OX40N     7.59
X200      1.94
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.88
X-MC    22.64
X-LC    20.21
M-SC    11.87
X-SC     8.27
H-MC     4.75
H-LC     3.10
L-SC     1.39
M-MC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.84,-4.68,38.53
IT,13.87,-14.16,73.10
FINANCE,9.47,-14.69,63.14
MISC,7.42,-22.69,72.95
ELECTRICAL,6.00,-9.38,49.45
PAINTS,5.93,-9.15,25.36
INSURANCE,4.41,-0.51,35.93
PHARMA,3.89,-0.55,32.73
AUTO,3.39,-18.51,69.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3077198.0,21
XR,1291370.0,13
AR,1278081.0,10
X40,1005851.0,14
X40N,730699.0,9
OX40N,710949.0,10
XY25,343010.0,6
SR,281396.0,2
MH,77211.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568679.0,25
M-SC,1365522.0,15
X-MC,1251385.0,16
X-LC,878340.0,11
X-SC,741390.0,8
H-MC,399646.0,3
L-SC,263282.0,3
M-LC,117952.0,1
H-LC,115291.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1223089.0      6
           AR         880514.0      5
M-SC       XY24       784756.0      6
H-SC       XR         776439.0      7
X-MC       X40        444963.0      7
X-LC       X40        388883.0      5
X-MC       XY24       376118.0      3
H-SC       OX40N      330030.0      4
M-SC       OX40N      293369.0      5
X-SC       X40N       292209.0      3
H-SC       SR         281396.0      2
X-SC       XY24       277176.0      3
X-MC       X40N       247459.0      4
X-LC       XY24       225458.0      2
H-MC       AR         209045.0      2
X-LC       X40N       191031.0      2
H-MC       XY24       190601.0      1
X-MC       XY25       182845.0      2
L-SC       XR         175732.0      2
X-SC       X40        172005.0      2
M-SC       XR         166933.0      2
           AR         120464.0      2
M-LC       XR         117952.0      1
L-SC       OX40N       87550.0      1
H-SC       MH          77211.0      1
X-LC       XY25        72968.0      2
H-LC       AR          68058.0      1
L-MC       XR          54314.0      1
M-MC       XY25        48287.0      1
H-LC       X200        47233.0      1
L-LC       XY25        38910.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
